In [1]:
from PyPDF2 import PdfReader, PdfWriter

def split_pdf(input_path, output_folder):
    """
    Разбивает PDF на отдельные страницы и сохраняет их как отдельные PDF-файлы.
    
    :param input_path: путь к исходному PDF-файлу
    :param output_folder: папка для сохранения результатов
    """
    reader = PdfReader(input_path)
    total_pages = len(reader.pages)

    print(f"Обнаружено страниц: {total_pages}")

    for i, page in enumerate(reader.pages, start=1):
        writer = PdfWriter()
        writer.add_page(page)

        output_filename = f"{output_folder}/page_{i:03d}.pdf"
        with open(output_filename, "wb") as output_file:
            writer.write(output_file)

        print(f"Сохранена страница {i} → {output_filename}")

# Пример использования
input_file = "input.pdf"
output_dir = "./split_pages"

import os
os.makedirs(output_dir, exist_ok=True)

split_pdf(input_file, output_dir)

ModuleNotFoundError: No module named 'PyPDF2'

In [2]:
pip install PyPDF2

   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/232.6 kB 262.6 kB/s eta 0:00:01
   ----- --------------------------------- 30.7/232.6 kB 262.6 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/232.6 kB 151.3 kB/s eta 0:00:02
   ------ -------------------------------- 41.0/232.6 kB 151.3 kB/s eta 0:00:02
   ---------- ---------------------------- 61.4/232.6 kB 182.2 kB/s eta 0:00:01
   ------------ -------------------------- 71.7/232.6 kB 196.9 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/232.6 kB 218.5 kB/s eta 0:00:01
   ------------------ ------------------- 112.6/232.6 kB 242.7 kB/s eta 0:00:01
   ------------------ ------------------- 112.6/232.6 kB 242.7 kB/s eta 0:

In [ ]:
from PyPDF2 import PdfReader, PdfWriter
import os

def parse_page_ranges(input_str, total_pages):
    """
    Парсит строку с номерами страниц, например:
    '1,3,5' → [1,3,5]
    '2-5,7' → [2,3,4,5,7]
    """
    selected_pages = []
    input_str = input_str.replace(" ", "")
    ranges = input_str.split(',')

    for r in ranges:
        if '-' in r:
            start, end = map(int, r.split('-'))
            if start > end:
                start, end = end, start
            selected_pages.extend(range(start, end + 1))
        else:
            selected_pages.append(int(r))

    # Удаляем дубликаты и проверяем диапазон
    unique_pages = sorted(set(selected_pages))
    valid_pages = [p for p in unique_pages if 1 <= p <= total_pages]

    return valid_pages

def extract_pages(input_path, pages, output_folder=None, single_file=False):
    reader = PdfReader(input_path)
    total_pdf_pages = len(reader.pages)

    if not output_folder:
        output_folder = os.path.dirname(input_path) or "."
    
    os.makedirs(output_folder, exist_ok=True)

    if single_file:
        writer = PdfWriter()
        for page_num in pages:
            writer.add_page(reader.pages[page_num - 1])

        output_path = os.path.join(output_folder, "selected_pages.pdf")
        with open(output_path, "wb") as f:
            writer.write(f)
        print(f"Сохранены страницы в один файл: {output_path}")
    else:
        for page_num in pages:
            writer = PdfWriter()
            writer.add_page(reader.pages[page_num - 1])
            output_path = os.path.join(output_folder, f"page_{page_num}.pdf")
            with open(output_path, "wb") as f:
                writer.write(f)
            print(f"Сохранена страница {page_num} → {output_path}")

def main():
    input_path = input("Введите путь к PDF-файлу: ").strip()
    if not os.path.isfile(input_path):
        print("❌ Файл не найден!")
        return

    reader = PdfReader(input_path)
    total_pages = len(reader.pages)
    print(f"Всего страниц в документе: {total_pages}")

    page_input = input("Введите номера страниц (например, 1,3 или 2-5): ").strip()
    selected_pages = parse_page_ranges(page_input, total_pages)

    if not selected_pages:
        print("❌ Не выбрано ни одной допустимой страницы.")
        return

    print(f"Будут извлечены страницы: {selected_pages}")

    single_choice = input("Сохранить в один файл? (y/n): ").strip().lower()
    single_file = single_choice in ('y', 'yes')

    output_folder = input("Папка для сохранения (оставьте пустой для текущей папки): ").strip() or None

    extract_pages(input_path, selected_pages, output_folder, single_file)

if __name__ == "__main__":
    main()